In [1]:
import pandas as pd

In [2]:
df_credits = pd.read_csv('./ml-kaggle/credits.csv')
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [3]:
df_links = pd.read_csv('./ml-kaggle/links.csv')
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
df_credits.columns = ['cast', 'crew', 'tmdbId']
df_credits = df_credits.merge(df_links, left_on='tmdbId', right_on='tmdbId')
df_credits = df_credits.drop(columns=['tmdbId', 'imdbId'])
df_credits.head()

,cast,crew,movieId
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",2
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",3
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",4
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",5


In [5]:
movies_valid_df = pd.read_csv('./ml-25m/movies_cleaned.csv')
valid_movies = movies_valid_df.movieId
movies_valid_df.head()

,movieId,title,genres,newId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1
2,3,Grumpier Old Men (1995),Comedy|Romance,2
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3
4,5,Father of the Bride Part II (1995),Comedy,4


In [6]:
actors = {}
directors = {}

acts = []
acts_mov = []
dirs = []
dir_mov = []

for i, row in df_credits.iterrows():
    movie_id = row.movieId
    cast = eval(row.cast)
    crew = eval(row.crew)
    for c in cast:
        actors[c['id']] = c['name']
        acts.append(c['id'])
        acts_mov.append(movie_id)
    for c in crew:
        if c['job'] == 'Director':
            directors[c['id']] = c['name']
            dirs.append(c['id'])
            dir_mov.append(movie_id)
    

In [7]:
actor_movie = pd.DataFrame(list(zip(acts, acts_mov)), columns=['actorId', 'movieId'])
director_movie = pd.DataFrame(list(zip(dirs, dir_mov)), columns=['directorId', 'movieId'])

In [8]:
len(actor_movie), len(director_movie)

(563112, 49120)

In [9]:
actor_movie = actor_movie.loc[actor_movie.movieId.isin(valid_movies)]
director_movie = director_movie.loc[director_movie.movieId.isin(valid_movies)]

In [10]:
actor_movie = actor_movie.groupby('actorId').filter(lambda df:df.shape[0] >= 10)
director_movie = director_movie.groupby('directorId').filter(lambda df:df.shape[0] >= 5)

valid_actors = actor_movie.actorId.unique()
valid_direct = director_movie.directorId.unique()

In [11]:
len(actor_movie), len(director_movie)

(62272, 6379)

In [12]:
actors = {key: actors[key] for key in valid_actors}
actor_df = pd.DataFrame(actors.items(), columns=['id', 'name'])
len(actor_df)

3446

In [13]:
directors = {key: directors[key] for key in valid_direct}
director_df = pd.DataFrame(directors.items(), columns=['id', 'name'])
len(director_df)

745

In [14]:
people_df = pd.concat([actor_df, director_df])
people_df = people_df.drop_duplicates()
len(people_df)

4132

In [15]:
people_df['peopleNewId'] = range(len(people_df))

In [16]:
with open('./files_big/people.txt', 'w') as f:
    for idx, row in people_df.iterrows():
        f.write(f'{row["name"]}\n')

In [17]:
movies_df = pd.read_csv('./ml-25m/movies_cleaned.csv')

In [18]:
movie_to_actor = {id: [] for id in range(len(movies_df))}

tmp_merged = movies_df.merge(actor_movie, left_on='movieId', right_on='movieId').merge(people_df, left_on='actorId', right_on='id')
print(tmp_merged.columns)
for newId in tmp_merged.newId.unique():
    movie_act_ids = list(tmp_merged.loc[tmp_merged.newId == newId].peopleNewId)
    movie_to_actor[newId] = movie_act_ids

Index(['movieId', 'title', 'genres', 'newId', 'actorId', 'id', 'name',
       'peopleNewId'],
      dtype='object')


In [19]:
with open('./files_big/movie_to_acts.txt', 'w') as f:
    lines = []
    for movie_id in range(len(movie_to_actor)):
        lines.append(f'{" ".join([str(t) for t in movie_to_actor[movie_id]])}\n')
    f.writelines(lines)

In [20]:
movie_to_dir = {id: [] for id in range(len(movies_df))}

tmp_merged = movies_df.merge(director_movie, left_on='movieId', right_on='movieId').merge(people_df, left_on='directorId', right_on='id')

for newId in tmp_merged.newId.unique():
    movie_act_ids = list(tmp_merged.loc[tmp_merged.newId == newId].peopleNewId)
    movie_to_dir[newId] = movie_act_ids

In [21]:
with open('./files_big/movie_to_directs.txt', 'w') as f:
    lines = []
    for movie_id in range(len(movie_to_dir)):
        lines.append(f'{" ".join([str(t) for t in movie_to_dir[movie_id]])}\n')
    f.writelines(lines)